# How many Reflect puzzles are there?

It's natural to wonder how many puzzles there are - in particular, are there enough "interesting" ones to keep publishing one every day?

## Counting Reflect boards mathematically

Let's think about boards first - ignoring the beams entirely for the moment.

Let's see how far we can get mathematically.

The number of ways of placing $p$ identical blocks (pieces) on an $N = n*n$ board is

$${N \choose p} = \frac{N!}{p!(N - p)!}$$

If there are $b$ types of block, then there are $b^p$ ways of choosing $p$ blocks (where order matters), and then the number of ways of placing $p$ blocks on the board is

$$\frac{{b^p}N!}{p!(N - p)!}$$

Take the following example. On an $N = 4*4 = 16$ board, with $p=3$ blocks, and $b=3$ types (represented by `/`, `\`, and `o`), the number of possible placements is

$$
\frac{{b^p}N!}{p!(N - p)!} = \frac{3^3 \cdot 16!}{3!13!}
                           = \frac{27 \cdot 16 \cdot 15 \cdot 14}{6}
                           = 15120
$$

We can calculate the total number of boards for increasing numbers of pieces with the following code:

In [1]:
from math import comb
import pandas as pd
rows = []
for num_pieces in range(1, 8):
    num_boards = comb(16, num_pieces) * pow(3, num_pieces)
    rows.append([num_pieces, num_boards])
pd.DataFrame(rows, columns=["Number of pieces", "Number of 4x4 boards"])

,Number of pieces,Number of 4x4 boards
0,1,48
1,2,1080
2,3,15120
3,4,147420
4,5,1061424
5,6,5837832
6,7,25019280


### Symmetries and equivalence classes

The previous formula for placements counts all boards, even those that would be considered the same under a transformation. For example, the second board below is the first board rotated through 180&deg;:

```
/../  ...\
....  ....
....  ....
\...  /../
```

A typical board belongs to an equivalence class of size eight (the symmetries are those of a square, called the dihedral group $D_4$). However, some have fewer in the class, such as this pair:

```
....  ....
./..  ..\.
../.  .\..
....  ....
```

The following board is an equivalence class of size one, since it in unchanged under the eight transformations:

```
....
./\.
.\/.
....
```

Since it is not easy to count these different cases mathematically, we turn to code to count them explicitly.

## Counting Reflect boards with code

Let's start by counting all the boards with a given number of pieces, just like we did above using a formula.

In [2]:
from reflect.count import *

boards3 = all_boards(3)
len(boards3)

15120

This tallies with the number from the formula above.

To understand how we can count boards that are the same under a transformation, take a look at the encoding for the boards. Each board is encoded as an 4-byte integer, with each piece on the board can be represented in 2 bits.

In [3]:
b = boards3[10000] # pick at random
b

536871170

In binary this looks like

In [4]:
bin(b)

'0b100000000000000000000100000010'

And if we stack it so it looks like a board

In [5]:
import re
val = bin(b)[2:].rjust(32, '0')
print("\n".join(re.findall('........', val)))

00100000
00000000
00000001
00000010


Using the `decode_board` function we can see what the board looks like, and the bit encoding for pieces (e.g. `\` is `10`):

In [6]:
print(decode_board(b).puzzle_solution())

......
..\...
......
..../.
....\.
......


The board encoding makes it easy to transform the board using efficient bit operations. We can *canonicalize* a board by applying all the symmetry transforms to it, and then picking the encoding with the lowest value:

In [7]:
canonicalize_board(b)

65632

In this case we can see that the canonicalized board is the original one rotated clockwise by 90&deg;.

In [8]:
print(decode_board(canonicalize_board(b)).puzzle_solution())

......
......
..../.
......
./\...
......


Using the `canonicalize_board` function, it's easy to see how many distinct boards with 3 pieces there are, taking symmetries into account

In [9]:
import numpy as mp
v_canonicalize_board = np.vectorize(canonicalize_board)
len(np.unique(v_canonicalize_board(boards3)))

1971

There is also a `canonical_boards` function that does the same thing, but more efficiently (using Numba).

In [10]:
len(canonical_boards(3))

1971

We can use this function to count the number of distinct boards for different numbers of pieces.

In [11]:
%%time
from math import comb
import pandas as pd
rows = []
for num_pieces in range(1, 8):
    num_boards = comb(16, num_pieces) * pow(3, num_pieces)
    assert num_boards == len(all_boards(num_pieces))
    rows.append([num_pieces, num_boards, len(canonical_boards(num_pieces))])
pd.DataFrame(rows, columns=["Number of pieces", "Number of 4x4 boards", "Number of distinct 4x4 boards"])

CPU times: user 11 s, sys: 6.11 s, total: 17.1 s
Wall time: 17.4 s


,Number of pieces,Number of 4x4 boards,Number of distinct 4x4 boards
0,1,48,9
1,2,1080,162
2,3,15120,1971
3,4,147420,18822
4,5,1061424,133569
5,6,5837832,732618
6,7,25019280,3132675


## Counting Reflect puzzles

So far, we've only considered block placement, and have ignored beams. A Reflect puzzle has a number of beams around the edges of the board, which may be chosen by the puzzle setter. This means that each board can correspond to a potentially large number of puzzles.

To simplify things, imagine that all beams have been arranged around the edge of the board. The question then becomes, does the puzzle have a unique solution? The answer is not always yes, as the following counterexample shows:

```
.EFBH.  .EFBH.
A....A  A....A
B../\H  B../.H
C.../G  C..\/G
D....D  D....D
.EFCG.  .EFCG.
```

It's also clear that _removing_ beams won't make the puzzle have a unique solution.

So how do we find the number of unique puzzles for a given number of pieces?

For every canonical board, we encode the pieces and the beams for that board and canonicalize them together. That allows us to find combinations of pieces and beams that have more than one associated board (i.e. puzzles that are *not* unique) - so we can filter those out to find the puzzles (pieces and beams) that *are* unique.

The code to do this is wrapped up in a function called `canonical_puzzles_with_unique_solution`. We can use it to find the number of unique 3 piece puzzles:

In [12]:
canonical_beams, canonical_pieces = canonical_puzzles_with_unique_solution(3)
len(canonical_beams), len(canonical_pieces)

(1541, 1541)

Summarizing for different numbers of pieces:

In [13]:
%%time
import pandas as pd
rows = []
for num_pieces in range(1, 8):
    canonical_beams, canonical_pieces = canonical_puzzles_with_unique_solution(
        num_pieces=num_pieces
    )
    assert len(canonical_beams) == len(canonical_pieces)
    rows.append([num_pieces, len(all_boards(num_pieces)), len(canonical_boards(num_pieces)), len(canonical_beams)])
pd.DataFrame(rows, columns=["Number of pieces", "Number of 4x4 boards", "Number of distinct 4x4 boards", "Number of distinct 4x4 puzzles with a unique solution"])

CPU times: user 1min 45s, sys: 9.46 s, total: 1min 55s
Wall time: 1min 55s


,Number of pieces,Number of 4x4 boards,Number of distinct 4x4 boards,Number of distinct 4x4 puzzles with a unique solution
0,1,48,9,9
1,2,1080,162,149
2,3,15120,1971,1541
3,4,147420,18822,11047
4,5,1061424,133569,49346
5,6,5837832,732618,135210
6,7,25019280,3132675,216008


For puzzles with 7 pieces, the vast majority of distinct boards (over 90%) are not unique. But even so, there are over 500 years of daily puzzles with 7 pieces!

## All the puzzles

Here's an idea: what we computed an index of all the possible puzzles just once? Then we could use the index to **count** the number of puzzles with certain properties (like uniqueness). We could also use it to **solve** puzzles by searching the index, or to **generate** puzzles by sampling the index.

Here's what it looks like:

In [14]:
duplicate_groups, board_index, beam_index, piece_index = all_puzzles(3)
len(duplicate_groups), len(board_index), len(beam_index), len(piece_index)

(15120, 15120, 15120, 15120)

The `all_puzzles` function returns four 1-D arrays, encoding the duplicate group, the boards, the beams, and the pieces for each puzzle. Each array has the same length, and the entries in the arrays at a given position correspond to each other, so `piece_index` will have the pieces for a particular board in `board_index`, for example.

Notice that the values are *not* canonicalized, we've got all transformations in this data. 

### Solving a puzzle

Let's use the index to solve this puzzle

In [15]:
beams = """
..CDA.
B.....
......
.....A
......
..CDB.
"""
pieces = ["\\", "\\", "\\"]
from reflect import Puzzle
puzzle = Puzzle.create(beams, pieces)

First we're going to use `duplicate_groups` to find all the puzzles with one solution.

In [16]:
single_solution_boards = board_index[duplicate_groups == 0]
single_solution_beams = beam_index[duplicate_groups == 0]
single_solution_pieces = piece_index[duplicate_groups == 0]
len(single_solution_boards), len(single_solution_beams), len(single_solution_pieces)

(11952, 11952, 11952)

Now we're going to filter by pieces to restrict to puzzles with just the pieces in the puzzle we are interested in:

In [17]:
puzzle.pieces_ints

array([2, 2, 2], dtype=int8)

In [18]:
pieces_val = encode_pieces_from_ints(puzzle.pieces_ints)
pieces_val

48

In [19]:
boards_with_pieces = single_solution_boards[single_solution_pieces == pieces_val]
beams_with_pieces = single_solution_beams[single_solution_pieces == pieces_val]
len(boards_with_pieces), len(beams_with_pieces)

(488, 488)

Let's look at one of the boards just to check it has the pieces we expect:

In [20]:
print(decode_board(boards_with_pieces[7]).puzzle_solution())

......
.\....
..\\..
......
......
......


It does, but they are not the solution to the puzzle, since we haven't taken beams into account.

We want to filter by beams, however, `beam_index` encodes for all beams in a puzzle, whereas the puzzle we are interested in does not have beams for all edges.

What we can do is use a bit mask to indicate which beams are encoded. The edge positions that have no beam will be set to 0 and masked out. This procedure finds all puzzles that have the beams that are set, while anything that is not set is a wildcard and is not constrained.

In [21]:
beams_val, beams_mask = encode_beams_from_puzzle(puzzle)
beams_index = (beams_with_pieces & beams_mask) == beams_val
matching_boards = boards_with_pieces[beams_index]
len(matching_boards)

1

We found one match! Here is the board, which is clearly the correct solution:

In [22]:
print(decode_board(matching_boards[0]).puzzle_solution())
print(beams)

......
.\....
......
.\..\.
......
......

..CDA.
B.....
......
.....A
......
..CDB.



The nice thing about this algorithm for solving a puzzle is that it turns a brute-force search that tries every possible combination of pieces on the board, into what is essentially a scan through a bunch of integers - which is much faster.

### Generating a puzzle

We can choose a random row from the index to generate a board that we could set as a puzzle:

In [23]:
from numpy.random import choice
val = choice(single_solution_boards)
board = decode_board(val)

# turn on all beams
for x, y in board.edge_locations():
    if board.values[y, x] == ".":
        board.add_beam(x, y)

print(board.puzzle_solution())

.ABCD.
E....E
Fo...J
G\...K
H..o.L
.GBID.


However, this puzzle has all beams around the edge, whereas we would like to set a more interesting puzzle that only has a subset of beams.

We can't just remove beams, as then the puzzle would not necessarily be unique.

What we can do though, is try turning the beam at each edge on or off and checking to see if the puzzle solution is unique using the solving algorithm above. We can then choose the puzzle with the fewest number of beams.

The maximum number of beams is 16, which works out at $2^{16}=65536$ beam patterns, but in practice there are usually many fewer (since mirror balls are usually few in number).

In [24]:
from reflect.generate import quick_minimise
b = quick_minimise(board)
print(b.puzzle_solution())

.A.C..
......
.o...J
G\....
H..o..
.G....


This is the basis of the algorithm I use to generate Reflect puzzles.